## __Measuring Ideal Gas Constant__ ##

Group: Logan MacHale, Jaden Majid

## Familiarization Notes - Feb 25 ##

__Clarify Objective__

We will be measuring the gas constant by assembling a force meter and using it to measure the force of the gas on various weights.

My hypothesis is that the gas constant for air can be measured using our apparatus and it will be similar to the scientifically accepted value.

__Physical meanings__

- Volume(V):$m^3$
- Force(F):$N$
- Pressure(P):$N/m^2$
- Mass(m): $kg$
- Acceleration(a): $m/s^2$
- Mols(n): $Dimensionless$

__Relate Quantities__

Think of all of the equations and tools you will need on the path from raw data to a final result.

What statistical tools do you need?

__What graphs will you do?__

1/V as a function of m


What other equations do I need?

- $V_{cylinder}=\pi*r^2*h$
- $A_{circle}=\pi*r^2$
- $P=F/A$
- $1/V=\frac{P_a}{nRT}-m\frac{g}{AnRT}$

__Logbook__

__15:05__ First run of data collection started

__15:45__ First run of data collection complete

__16:00__ Friction must be taken into account

## Mar 1 ##

__Equation with friction:__ $\dfrac{1}{V} = -m * \dfrac{g}{AnRT}+\dfrac{AP_a+F_k}{AnRT}$

__Logbook:__

__15:15__ second run of data collection started